In [1]:
import tools

import Bio
from Bio.PDB import PDBParser
from Bio.PDB.Chain import Chain

import py3Dmol

import warnings

from typing import List

import json

In [2]:
# Load example, seeing interactions between A & B chain
warnings.filterwarnings("ignore", category=Bio.PDB.PDBExceptions.PDBConstructionWarning)

parser = PDBParser()
structure = parser.get_structure("9fg1", "9fg1.pdb")

structure

<Structure id=9fg1>

In [ ]:
# Get A & B chains
A: Chain = structure[0]['A']
B: Chain = structure[0]['B']

In [ ]:
# View A & B chains
with open('9fg1.pdb', 'r') as protien:

  pdb_text = protien.read()

view = py3Dmol.view(width=800, height=800)

view.addModel(pdb_text)

view.setStyle({'cartoon': {'color': 'black'}})
view.setStyle({'chain': 'A'}, {'cartoon': {'color': 'red'}})
view.setStyle({'chain': 'B'}, {'cartoon': {'color': 'blue'}})

view.zoomTo()
view.show()

In [4]:
# Return class structure for refrence
class AtomicID:
    """Identifier for molecules in the protien"""
    chain: str

    residue_number: int
    residue_type: str

    atom_type: str
    atom_number: int

class Interaction:

  id: str
  global_distance: float

  # If using by closest atomic interaction
  closest_atomic_diffence: float | None = None
  self_closest: AtomicID | None = None
  other_closest: AtomicID | None = None

class InteractingResidue:

  chain: str
  position: int # in chain

  # For later analysis, in 3 letter format
  residue: str

  # to keep track of interactions
  id: str

  interactions: List[Interaction]

In [4]:
# Get interacting residues
# pseudonyms ribd, by_residue_distance, residue_interaction_by_distance
interacting_residues: List[tools.distance.InteractingResidue] = tools.by_residue_ca(A, B, 10.0)

In [5]:
# Get InteractingResidue from id
aa = interacting_residues[0]

print(tools.residue_by_id(interacting_residues, aa.interactions[0].id))

InteractingResidue(chain='B', position=30, residue='ASP', id='B:30', interactions=[Interaction(id='A:11', global_distance=7.791790008544922, hash=7047801719230248770, closest_atomic_diffrence=None, self_closest=None, other_closest=None), Interaction(id='A:12', global_distance=7.651941299438477, hash=-1728674844451608096, closest_atomic_diffrence=None, self_closest=None, other_closest=None)])


In [ ]:
# By closest atomic distance
interacting_residues: List[tools.distance.InteractingResidue] = tools.residue_closest_atomic_interaction(A, B, 10.0)

In [8]:
# Show interacting residues

# view from before
for residue in interacting_residues:

    view.setStyle(
        {'chain': residue.chain, 'resi': residue.position},
        {'cartoon': {'color': 'yellow'}}
    )

view.zoomTo()
view.show()

In [5]:
# To json
with open('9fg1.test.json', 'w') as save_file:

    json_data = [i.to_json() for i in interacting_residues]

    json.dump(json_data, save_file)

In [6]:
# From json
with open('9fg1.test.json', 'r') as save_file:

    json_data = json.load(save_file)

    jl = tools.distance.InteractingResidue.from_json

    data = [jl(r) for r in json_data]

data

[InteractingResidue(chain='A', position=11, residue='ASN', id='A:11', interactions=[Interaction(id='B:30', global_distance=7.791790008544922, closest_atomic_diffence=6.638417720794678, self_closest=AtomicID(chain='A', residue_number=11, residue_type='ASN', atom_type='C', atom_number=4), other_closest=AtomicID(chain='B', residue_number=30, residue_type='ASP', atom_type='C', atom_number=1)), Interaction(id='B:31', global_distance=7.525432586669922, closest_atomic_diffence=4.741662502288818, self_closest=AtomicID(chain='A', residue_number=11, residue_type='ASN', atom_type='O', atom_number=3), other_closest=AtomicID(chain='B', residue_number=31, residue_type='PHE', atom_type='O', atom_number=3)), Interaction(id='B:32', global_distance=7.761246681213379, closest_atomic_diffence=6.531132698059082, self_closest=AtomicID(chain='A', residue_number=11, residue_type='ASN', atom_type='C', atom_number=4), other_closest=AtomicID(chain='B', residue_number=32, residue_type='GLY', atom_type='O', atom_n